## Importing Packages

In [ ]:
import xarray as xr
import pandas as pd
import numpy  as np

from eofs.xarray import Eof
from scipy.fft import rfft, rfftfreq

import netCDF4
from netCDF4 import Dataset

import geopandas as gpd
import rioxarray as rio
from shapely.geometry import mapping, Polygon

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import rcParams

import sys
import subprocess

import seaborn as sns

import datetime

import glob
import os

import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.style.use('seaborn-darkgrid')
rcParams['font.family'] = 'monospace'
rcParams['font.sans-serif'] = ['Lucida Console']

In [ ]:
def annot_max(x, y, ax=None):
    ymax = np.argsort(y)[-3:][::-1]
    xmax = x[ymax]
    for i in np.arange(0,3):
        text = f"x={xmax[i]:.1f}, y={y[ymax[i]]:.1e}"
        if not ax:
            ax=plt.gca()
        bbox_props = dict(boxstyle="round,pad=0.3", fc="w", ec="darkred", lw=0.72)
        # arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60,rad=5")
        kw = dict(xycoords='data',textcoords="axes fraction",
                bbox=bbox_props, ha="right", va="top")#arrowprops=arrowprops, 
        ax.annotate(text, xy=(xmax[i], y[ymax[i]]),  xytext=(0.94, 0.95-i/4), **kw)

## Dataset

#### Map Information

In [ ]:
shp = 'd:/00_Masters/01_Dados/Shapes/brasil_UF.shp'

extent = [-39.1,-33, -14.3,-4.3] # lonmin lonmax latmin latmax
proj = ccrs.PlateCarree()

shp_tomask = 'd:/00_Masters/01_Dados/Shapes/Buffer1GRAU_paraCORTE.shp'
geodf = gpd.read_file(shp_tomask)
geodf.set_crs("epsg:4326",inplace=True)

#### Glorys Dataset -  First Time Processing

In [ ]:
# # Get a list of all .nc files available in different folders
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
dsmerged = xr.open_mfdataset(filenames)

# # Corrige erro de datas duplicadas pelo mfdataset
_, index = np.unique(dsmerged['time'], return_index=True)
dsmerged = dsmerged.isel(time=index)

             Defining DEPTHS of interest 

In [ ]:
Depth_0 = dsmerged.sel(depth=0,method='nearest')
temp_0 = Depth_0['thetao']
clim_0 = temp_0.mean('time')#,skipna=True)

Depth_30 = dsmerged.sel(depth=30,method='nearest')
temp_30 = Depth_30['thetao']
clim_30 = temp_30.mean('time')#,skipna=True)

depths = [0,30]

###### REMOVING SEASONAL CICLE

In [ ]:
Means_0 = temp_0.groupby("time.dayofyear").mean()
NoSeason_0 = temp_0.groupby("time.dayofyear") - Means_0
NoSeason_0 = NoSeason_0.to_dataframe()
NoSeason_0 = NoSeason_0['thetao'].to_xarray()

Means_30 = temp_30.groupby("time.dayofyear").mean()
NoSeason_30 = temp_30.groupby("time.dayofyear") - Means_30
NoSeason_30 = NoSeason_30.to_dataframe()
NoSeason_30 = NoSeason_30['thetao'].to_xarray()

ds_list = [NoSeason_0, NoSeason_30]
ds_strings = ['0.49m','29.44m']

             Exporting data to NC for better perfomance

In [ ]:
NoSeason_0.to_netcdf('d:/00_Masters/01_Dados/NoSeason_0.nc')
NoSeason_30.to_netcdf('d:/00_Masters/01_Dados/NoSeason_30.nc')

# EOF

##### Importing data from new NC for better performance

In [ ]:
NoSeason_0 = xr.open_dataset('d:/00_Masters/01_Dados/NoSeason_0.nc')
NoSeason_30 = xr.open_dataset('d:/00_Masters/01_Dados/NoSeason_30.nc')

depths = [0,30]
ds_list = [NoSeason_0['thetao'], NoSeason_30['thetao']]
ds_strings = ['0.49m','29.44m']

# # Adiciona referencia geospacial para corte
for ds,i in zip(ds_list,range(0,len(depths))):
    ds.rio.set_spatial_dims("longitude","latitude",inplace=True)
    ds.rio.write_crs("epsg:4326",inplace=True)

    ds_list[i] = ds_list[i].rio.clip(geodf.geometry.apply(mapping), geodf.crs, drop=False, invert=False)     # clips everything - all nan

In [ ]:
"""
Compute and plot the leading EOF of sea surface temperature (SST) in the
west Atlantic.

This routine uses the metadata-retaining xarray interface.

Additional requirements for this example:
    * xarray (http://xarray.pydata.org)
    * matplotlib (http://matplotlib.org/)
    * cartopy (http://scitools.org.uk/cartopy/)
"""

percent = {}
n = 5 # set number of EOFs and PCs to calculate
period = 1 #24*60*60 - período amostral em segundos
sample_freq = 1/period
Nyquist = sample_freq/2


# Read SST anomalies Dataset.
for ds,st,d in zip(ds_list,ds_strings,depths):
    sst = ds

    # Create an EOF solver to do the EOF analysis. Square-root of cosine of
    # latitude weights are applied before the computation of EOFs.
    coslat = np.cos(np.deg2rad(sst.coords['latitude'].values))
    wgts = np.sqrt(coslat)[..., np.newaxis]
    solver = Eof(sst, weights=wgts,center=True)

    # Retrieve the leading EOF, expressed as the correlation between the leading
    # PC time series and the input SST anomalies at each grid point, and the
    # leading PC time series itself.
    eof1 = solver.eofsAsCorrelation(neofs=n)
    pc1 = solver.pcs(npcs=n, pcscaling=1)

    # Retrieve the Leading EOF representation, expressed as percent
    percent['Depth_'+str(d)] = Eof.varianceFraction(solver,neigs=n)

    # Plot the leading EOF expressed as correlation in the domain.
    for i in range(0,n,1):

        # Retrieve FFT from PC time series
        Nobs = len(pc1[:,i])
        sig_fft = rfft(pc1[:,i].values,Nobs)
        power = np.abs(sig_fft)**2
        sig_fftfreqs = rfftfreq(Nobs,d=sample_freq)
        per_fftfreqs=(1./sig_fftfreqs)/365

        # FIGURE
        fig = plt.figure(figsize=(9.4,4.2)) #figsize=(3.4,5.8)
        gs = fig.add_gridspec(3,6)

        fig.subplots_adjust(left=0.15, bottom=0.1, right=.8, top=0.95, hspace=.5, wspace=1.75)
        fig.suptitle('Depth = '+str(d)+'m', fontsize=12, y=1.05, x=.45,horizontalalignment='center')
        fig.patch.set_facecolor('white')

        # # Plot the EOF - MAP Subplot
        ax = fig.add_subplot(gs[:,:3], projection=proj)
        ax.set_extent(extent)

        gl = ax.gridlines(crs=proj, draw_labels=True,
                          linewidth=.5, color='gray', alpha=0.5, linestyle='--')
        gl.xlocator = mticker.FixedLocator(np.arange(-34,-39.5,-2))
        gl.ylocator = mticker.FixedLocator(np.arange(-5,-14.5,-2.5)) 
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        plt.rcParams.update({'font.size': 8})
        
        clevs = np.linspace(-1,1,100)
        fill = eof1[i].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                            add_colorbar=False, transform=ccrs.PlateCarree())
        
        shape_feature = ShapelyFeature(Reader(shp).geometries(),
                                        ccrs.PlateCarree(), edgecolor='gray',linewidth=.5)
        
        ax.add_feature(shape_feature,facecolor='oldlace')

        divider = make_axes_locatable(ax)
        ax_cb  = divider.new_horizontal(size="5%", pad=0.075, axes_class=plt.Axes)
        cbar_ax = fig.add_axes(ax_cb)
        cb = plt.colorbar(fill, cax= cbar_ax, orientation='vertical',) #shrink=0.8
        cb.set_ticks([-1,0,1])
        cb.set_label('Corr. Coefficient', fontsize=8,labelpad=-2)
        ax.set_title('EOF' + str(i+1) + ' - ' + str((
            percent['Depth_'+str(d)][i].values*100).round(2)) + '%', fontsize=9)
        ax.tick_params(axis='both',labelsize=8)

        # # Plot the leading PC time series.
        ax2 = fig.add_subplot(gs[:2,3:])
        pc1[:, i].plot(color='darkblue', linewidth=1)
        ax2 = plt.gca()
        ax2.axhline(0, color='k',alpha=0.5,linewidth=0.5)
        ax2.grid(b=True, which='major', color='w', linewidth=0.5)
        ax2.set_ylim(-5, 5)
        ax2.set_yticks(np.arange(-4,5,1))
        ax2.set_ylabel('Normalized Units',fontsize=8)
        ax2.set_xlabel('Year',fontsize=8,loc='right',labelpad = -5)
        ax2.set_title('PC' + str(i+1) + ' Time Series', fontsize=9)
        ax2.autoscale(enable=True, axis='x', tight=True)
        ax2.tick_params(axis='both',labelsize=8)
        ax2.tick_params(axis='x',pad=-.5)
        for label in ax2.get_xticklabels(which='major'):
            label.set(rotation=25, ha='right', rotation_mode='anchor')

        # # Plot FFT
        ax3 = fig.add_subplot(gs[2:,3:])
        # ax3.plot(per_fftfreqs,power,color='darkred', linewidth=1)#/(10**-6)
        plt.stem(per_fftfreqs,power,linefmt='darkred',markerfmt=" ",basefmt=None,use_line_collection=True)
        ax3 = plt.gca()
        ax3.grid(b=True, which='major', color='w', linewidth=0.5)
        ax3.set_xlim(0,18)
        ax3.set_ylabel('Power',fontsize=8)
        ax3.set_xlabel('Period (years)',fontsize=8)
        ax3.set_title('FFT' + str(i+1), fontsize=9)
        ax3.tick_params(axis='both',labelsize=8)
        ax3.ticklabel_format(axis='y',style='sci',useMathText=True,scilimits=(0,0))
        annot_max(per_fftfreqs,power,ax=ax3)
        
        plt.savefig(
            'D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_Temp/Figuras/EOF'+str(i+1)+'_'+str(d)+'.png',dpi=400, bbox_inches='tight'
            )
        plt.close()

        tmp = pd.DataFrame({'Power':power[:],'Amplitude':sig_fft[:],'Cycles/day':sig_fftfreqs[:]})
        tmp['Period [days]'] = np.round(((per_fftfreqs))/sample_freq,2)
        tmp.sort_values(by=['Amplitude'], ascending=False).head(20).to_csv(
            'D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_Temp/Outputs/EOF'+str(i+1)+'_'+str(d)+'.csv',sep=';', index=False
            )

------